# Iterative Model Trainer



## Training Progress

#### Iterations Completed
 * 1 - 350 Iterations Completed
 * 2001 - 2075 Iterations Completed

In [ ]:
!pip install transformers datasets sacrebleu rouge_score torch accelerate -q -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 65.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 66.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 91.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.

In [ ]:
from transformers import pipeline
from datasets import load_dataset, Dataset, DatasetDict
from transformers import AutoTokenizer, BartForConditionalGeneration, DataCollatorForSeq2Seq
from transformers import TrainingArguments, Trainer
from google.colab import drive
import torch
import random
import pandas as pd
import re

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
# for first time
# model_path = "facebook/bart-large-cnn"

# for nth iteration
model_path = "./drive/MyDrive/Submission/BE Project Group No 31/Model/iter_trained_model"

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
model = BartForConditionalGeneration.from_pretrained(model_path).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
i = 326

train_df = pd.DataFrame()
val_df = pd.DataFrame()

for current_data_iteration in range(i,i+25):
  train_dataset_path = f"./drive/MyDrive/Submission/BE Project Group No 31/Data/training_data{current_data_iteration}.csv"
  val_dataset_path = f"./drive/MyDrive/Submission/BE Project Group No 31/Data/val_data{current_data_iteration}.csv"
  train_df = pd.concat([train_df, pd.read_csv(train_dataset_path)])
  val_df = pd.concat([val_df, pd.read_csv(val_dataset_path)])


train_dataset = Dataset.from_pandas( train_df )
val_dataset = Dataset.from_pandas( val_df )
papers_dataset = DatasetDict({"train":train_dataset, "validation":val_dataset})

In [ ]:
def remove_section_headers(text):
    # Define the regular expression pattern to match sequences like "1. xy"
    section_patterns = [
        # Digits
        r'\b\d+\.\s+\w+\b',
        # Roman Numerals
        r'\b[IVXLCDM]+\.\s+\w+\b'
    ]

    # Combine patterns into a single regex pattern
    pattern = '|'.join(section_patterns)

    # Use re.sub to replace matched sequences with an empty string
    cleaned_text = re.sub(pattern, '', text)

    return cleaned_text

def remove_citations(text):
    # Define patterns for common citation formats
    citation_patterns = [
        # Match citations like "[1]", "[12]", "[123]", etc.
        r'\[\d+\]',

        # Match citations like "(Author, Year)", "(Author et al., Year)", "(Author Year)", etc.
        r'\(\w+(?: et al.)?, \d{4}\)',

        # Match citations like "[Author et al., Year]", "[Author, Year]", etc.
        r'\[\w+(?: et al.)?, \d{4}\]'
    ]

    # Combine patterns into a single regex pattern
    combined_pattern = '|'.join(citation_patterns)

    # Remove citations from the text using the regex pattern
    cleaned_text = re.sub(combined_pattern, '', text)

    return cleaned_text

def replace_multiple_whitespace(text):
    # Define the regular expression pattern to match multiple whitespace characters
    pattern = r'\s+'

    # Use re.sub to replace multiple whitespace characters with a single whitespace character
    cleaned_text = re.sub(pattern, ' ', text)

    return cleaned_text

def fix_grammar(text):
    tool = LanguageTool('en-US')

    # Check for grammatical errors
    matches = tool.check(text)
    # Fix grammatical errors
    corrected_text = tool.correct(text)

    return corrected_text

def preprocess_text(text):
    # replace new line character with space
    text = text.replace("\n"," ")

    # removing extra whitespaces
    text = replace_multiple_whitespace(text)

    # remove citations
    text = remove_citations(text)

    #remove section heading
    text = remove_section_headers(text)

    return text

def preprocess_dataset(dataset_dict, func):
    for dataset_name, dataset in dataset_dict.items():
        dataset_dict[dataset_name] = dataset.map(lambda example: {
            'article': func(example['article']),
            'abstract': func(example['abstract'])
        })

    return dataset_dict

papers_dataset = preprocess_dataset(papers_dataset, preprocess_text)

Map:   0%|          | 0/2550 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
def tokenize_function(examples):
    input_encodings = tokenizer(examples["article"], max_length=1024, padding="max_length", truncation=True, return_tensors="pt")

    with tokenizer.as_target_tokenizer():
      target_encodings = tokenizer(examples["abstract"],max_length=128, padding="max_length", truncation=True, return_tensors="pt")

    return {
        "input_ids" : input_encodings["input_ids"],
        "attention_mask" : input_encodings["attention_mask"],
        "labels" : target_encodings["input_ids"]
    }

papers_dataset_processed = papers_dataset.map(tokenize_function,batched=True)

Map:   0%|          | 0/2550 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3892: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
papers_dataset_processed

DatasetDict({
    train: Dataset({
        features: ['article', 'abstract', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 2550
    })
    validation: Dataset({
        features: ['article', 'abstract', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 100
    })
})

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer,model=model)

In [ ]:
training_args = TrainingArguments(
    output_dir="./bart-papers-trained-output",
    per_device_train_batch_size=1,
    save_total_limit=2,
    num_train_epochs=2,
    evaluation_strategy="steps",
    eval_steps=250,
    save_steps=1000,
    logging_dir="./logs",
    logging_steps=100,
    remove_unused_columns=True,  # Set to True to remove extra columns in the dataset
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=papers_dataset_processed["train"],
    eval_dataset=papers_dataset_processed["validation"],
)

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss
250,2.548700,2.476507
500,2.394800,2.454811
750,2.521900,2.474125
1000,2.587700,2.479284
1250,2.492200,2.481658
1500,2.382400,2.458479
1750,2.411200,2.456498
2000,2.331500,2.448721
2250,2.235200,2.450222
2500,2.305000,2.422759


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_

TrainOutput(global_step=5100, training_loss=2.0769088086894913, metrics={'train_runtime': 3748.0445, 'train_samples_per_second': 1.361, 'train_steps_per_second': 1.361, 'total_flos': 1.10522334707712e+16, 'train_loss': 2.0769088086894913, 'epoch': 2.0})

In [ ]:
trainer.save_model("./drive/MyDrive/Submission/BE Project Group No 31/Model/iter_trained_model")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


In [ ]:
print(f"{current_data_iteration}th Iteration Training Completed")

350th Iteration Training Completed
